# Implicit Hate Speech Classification

Import all necessary libraries and install everything you need for training:

First, enable the GPU - under Accelerator on the right of the site, choose GPU. Be careful to always terminate the session (click the power off button), otherwise it will still be running and you will lose the 30 hours of GPU that you have available per week.

In [ ]:
# install the libraries necessary for data wrangling, prediction and result analysis
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix, f1_score
import torch
from numba import cuda
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier

In [ ]:
# Install transformers
# (this needs to be done on Kaggle each time you start the session)
!pip install -q transformers

In [ ]:
# Install the simpletransformers
!pip install -q simpletransformers
from simpletransformers.classification import ClassificationModel

### Import the data
You might need to upload the data (click on the Add data button on the left of the site). I have uploaded the first version of the data that I created (see the 1-Data-Preparation.ipynb spreadsheet): "hatespeechdataset". If you change the Google Sheet, you can reprocess it by running the data preparation spreadsheet and upload the new version of it (go to the dataset description (https://www.kaggle.com/datasets/tajakuz/hatespeechdataset), click on the three dots and choose "New Version)

In [ ]:
# Upload the implicit dataset, prepared for the Simple Transformers
impl_dataset = pd.read_csv("/kaggle/input/hatespeechdataset/implicitness_binary_dataset.csv", sep="\t", index_col=0)
impl_dataset.head()

In [ ]:
# See the statistics on the dataset
impl_dataset.describe()

In [ ]:
# Define the labels
LABELS = [0,1]

In [ ]:
# First, let's split the implicit dataset into train and test split (70:30) by using the train_test_split from the sci-kit learn. We will shuffle the data beforehand and stratify it according to the labels (so that the distribution of labels is the same in both splits as in the original dataset)

impl_train, impl_test = train_test_split(impl_dataset, test_size=0.3, random_state=42, shuffle = True, stratify = impl_dataset.labels)

# See the size of the splits
impl_train.shape, impl_test.shape


In [ ]:
# Check how the splits look like
impl_train.head(3)

In [ ]:
# Check how the splits look like
impl_test.head(3)

In [ ]:
# Create a file to save results into (you can find it under Data: Output). Be careful, run this step only once to not overwrite the results file.
results = []

with open("Implicit-Experiments-Results.json", "w") as results_file:
    json.dump(results,results_file, indent= "")

In [ ]:
# In each next step (after the first experiment), open the results file instead of creating a new results file:
with open("./Implicit-Experiments-Results.json", "r") as results_file:
    previous_results = json.load(results_file)

# See the results
previous_results

## Training and testing - dummy classifier

Let's first apply a baseline classifier which predicts the most frequent class to each instance, to see what is the baseline score.

In [ ]:
# Create X_train and Y_train parts, used for sci kit learning
# We need to split each split (test and train) into an object with just texts and object with just labels
X_train = list(impl_train.text)
Y_train = list(impl_train.labels)

X_test = list(impl_test.text)
Y_test = list(impl_test.labels)

# See their sizes
len(X_train), len(Y_train), len(X_test), len(Y_test)

In [ ]:
# Use the Dummy Classifier, with the strategy "most_frequent"
dummy_clf = DummyClassifier(strategy="most_frequent")

# Train the model
dummy_clf.fit(X_train, Y_train)

#Get the predictions
y_pred = dummy_clf.predict(X_test)

In [ ]:
# Compare the predictions with true values (Y_test)
micro = f1_score(Y_test, y_pred, labels=LABELS, average ="micro")
macro = f1_score(Y_test, y_pred, labels=LABELS, average ="macro")
accuracy = round(metrics.accuracy_score(Y_test, y_pred),3)
print(f"Micro F1: {micro:.3f}, Macro F1: {macro:.3f}, Accuracy: {accuracy}")

In [ ]:
# Save the results:
rezdict = {
    "model": "dummy",
    "microF1": micro,
    "macroF1": macro,
    "accuracy": accuracy,
    "y_pred": f"{y_pred.tolist()}"
    }
previous_results.append(rezdict)

In [ ]:
previous_results

## Training and testing - Transformer model

We will use the basic English monolingual BERT model: https://huggingface.co/bert-base-uncased

You can find more documentation on how to use Simple Transformer models here: https://simpletransformers.ai/docs/usage/

For the hyperparameters (args), I used the ones that worked for me before, but you can see the entire list here: https://simpletransformers.ai/docs/usage/#configuring-a-simple-transformers-model

In [ ]:
# Define the model
bertbase_model = ClassificationModel(
        "bert", "bert-base-cased",
        num_labels=2,
        use_cuda=True,
        args= {
    "num_train_epochs": 60,
    "labels_list": LABELS,
    "learning_rate": 1e-5,
    # We'll use a smaller max_seq_length (we could set it up to 512), because we have short texts
    "max_seq_length": 128,
    # Use this to mute the long output that tells you how the model proceeds.
    "silent": True,
    # Below are just some additional hyperparameters that we found that help with memory errors
    "save_steps": -1,
    "overwrite_output_dir": True,
    "no_cache": True,
    "no_save": True,
    }
    )

In [ ]:
# Train the model on train data - this will take some time
bertbase_model.train_model(impl_train)

print("Training is finished!")

In [ ]:
# Test the model - this will take some time

# Get the true labels
y_true = impl_test.labels

# Calculate the model's predictions on test
def make_prediction(input_string):
    return bertbase_model.predict([input_string])[0][0]

y_pred = impl_test.text.apply(make_prediction)

print("Testing is finished!")

In [ ]:
# Calculate the scores
macro = f1_score(y_true, y_pred, labels=LABELS, average="macro")
micro = f1_score(y_true, y_pred, labels=LABELS,  average="micro")
accuracy = round(metrics.accuracy_score(y_true, y_pred),3)
print(f"Macro f1: {macro:0.3}, Micro f1: {micro:0.3}, Accuracy: {accuracy}")

In [ ]:
# Plot the confusion matrix:
cm = confusion_matrix(y_true, y_pred, labels=LABELS)
plt.figure(figsize=(9, 9))
plt.imshow(cm, cmap="Oranges")
for (i, j), z in np.ndenumerate(cm):
    plt.text(j, i, '{:d}'.format(z), ha='center', va='center')
#classNames = LABELS
classNames = ["Explicit", "Implicit"]
plt.ylabel('True label')
plt.xlabel('Predicted label')
tick_marks = np.arange(len(classNames))
plt.xticks(tick_marks, classNames, rotation=90)
plt.yticks(tick_marks, classNames)
plt.title("Binary Implicit vs. Explicit Hate Speech Classification")

plt.tight_layout()
fig1 = plt.gcf()
plt.show()
plt.draw()
fig1.savefig(f"Confusion-matrix-implicit-classification.png",dpi=100)

In [ ]:
# Save the results:
rezdict = {
    "model": "BERT",
    "microF1": micro,
    "macroF1": macro,
    "accuracy": accuracy,
    "y_pred": f"{y_pred.tolist()}"
    }
previous_results.append(rezdict)

#Save intermediate results (just in case)
backup = []
backup.append(rezdict)
with open(f"backup-results.json", "w") as backup_file:
    json.dump(backup,backup_file, indent= "")

In [ ]:
# Compare the results by creating a dataframe from the previous_results dictionary:
results_df = pd.DataFrame(previous_results)

results_df

We can see that BERT performs better than the baseline.

In [ ]:
# Print the dataframe in a markdown format:
print(results_df.drop(columns="y_pred").to_markdown())

In [ ]:
# Add the end, save the file with results:
with open("./Implicit-Experiments-Results.json", "w") as final_results_file:
    json.dump(previous_results,final_results_file, indent= "")

In [ ]:
# Add the information about the predictions to the main table with information about implicitness

# Open the main table
main_sheet = pd.read_csv("/kaggle/input/hatespeechdataset/hate-speech-prepared-spreadsheet.csv", sep="\t", index_col = 0)

main_sheet.head()

In [ ]:
# Add the information about predictions to the main sheet

main_sheet["implicit-y_pred"] = y_pred

# Add also the labels, converted to integers
main_sheet["implicit-y_true"] = y_true

main_sheet.head()

In [ ]:
# Save the extended sheet
main_sheet.to_csv("hate-speech-prepared-spreadsheet-implicit-prediction.csv", sep = "\t")